In [1563]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import math
import numpy as np
import pandas as pd

In [1564]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [1565]:
song_meta = pd.read_json('../res/song_meta.json', typ = 'frame')

In [1566]:
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [1567]:
song_meta_indexed = song_meta.set_index('id')

In [1568]:
song_artist = []

def get_song_artist(song, artist_list):
    for artist in artist_list:
        if artist is not 'Various Artists':
            song_artist.append([song, artist])

In [1569]:
result = song_meta.apply(lambda x : get_song_artist(x.id, x.artist_name_basket), axis=1)

In [1570]:
song_artist_df = pd.DataFrame(song_artist, columns=['song', 'artist'])

In [1571]:
song_artist_df = song_artist_df.set_index('song')

In [1572]:
song_album_df = song_meta[['id', 'album_id']]

In [1573]:
song_album_df.columns = ['song', 'album']

In [1574]:
song_album_df =  song_album_df.set_index('song')

In [1575]:
song_gn = []

def get_song_gn(song, gn_list):
    if len(gn_list) == 0:
        song_gn.append([song, 'nogn'])
    else:
        for gn in gn_list:
            song_gn.append([song, gn])

In [1576]:
result = song_meta.apply(lambda x : get_song_gn(x.id, x.song_gn_dtl_gnr_basket), axis=1)

In [1577]:
song_gn_df = pd.DataFrame(song_gn, columns=['song', 'gn'])

In [1578]:
song_gn_df = song_gn_df.set_index('song')

In [1579]:
train = pd.read_json('../res/train.json', typ = 'frame')

In [1580]:
val = pd.read_json('../res/val.json', typ = 'frame')

In [1581]:
test = pd.read_json('../res/test.json', typ = 'frame')

In [1582]:
train['songs_len'] = train['songs'].apply(lambda x : len(x))
train['tags_len'] = train['tags'].apply(lambda x : len(x))
val['songs_len'] = val['songs'].apply(lambda x : len(x))
val['tags_len'] = val['tags'].apply(lambda x : len(x))
test['songs_len'] = test['songs'].apply(lambda x : len(x))
test['tags_len'] = test['tags'].apply(lambda x : len(x))

In [1583]:
model = Word2Vec.load("song2vec.model")

In [1584]:
song_song_list = []

In [1585]:
song_song_count = 0

In [1586]:
def co_occur_song(song_list):
    global song_song_count
    if song_song_count % 10000 == 0:
        print(f'===== working at {song_song_count},  {datetime.now()} =====')
    for idx1,song1 in enumerate(song_list):
        for idx2,song2 in enumerate(song_list[idx1:idx1+30]):
            if song1 is not song2:
                song_song_list.append([song1, song2])
                song_song_list.append([song2, song1])
    song_song_count = song_song_count + 1
            

In [1587]:
result = train['songs'].apply(co_occur_song)

===== working at 0,  2020-07-22 12:46:38.211735 =====
===== working at 10000,  2020-07-22 12:46:46.702655 =====
===== working at 20000,  2020-07-22 12:47:02.715571 =====
===== working at 30000,  2020-07-22 12:47:13.530866 =====
===== working at 40000,  2020-07-22 12:47:16.561120 =====
===== working at 50000,  2020-07-22 12:47:28.457626 =====
===== working at 60000,  2020-07-22 12:47:41.741932 =====
===== working at 70000,  2020-07-22 12:47:44.650997 =====
===== working at 80000,  2020-07-22 12:47:59.500518 =====
===== working at 90000,  2020-07-22 12:48:02.474688 =====
===== working at 100000,  2020-07-22 12:48:19.404756 =====
===== working at 110000,  2020-07-22 12:48:22.317207 =====


In [1588]:
co_song_df = pd.DataFrame(song_song_list, columns=['sa', 'sb'])

In [1589]:
co_song_df.shape

(215131424, 2)

In [1590]:
co_song_count_df = co_song_df.groupby(['sa', 'sb']).size().reset_index().set_index('sa')

In [1591]:
co_song_count_df.shape

(138395578, 2)

In [1592]:
co_song_count_df.columns = ['sb', 'count']

In [1593]:
co_song_count_df['count_scale'] = co_song_count_df['count'].apply(lambda x : math.log(x+1))

In [1594]:
pp_song_count_df = co_song_df.groupby('sa').size().sort_values(ascending=False).reset_index().set_index('sa')

In [1595]:
tag_song_list = []

In [1596]:
def co_occur_tag_song(song_list, tag_list):
    tags = list(set(tag_list))
    for tag in tags:
        for song in song_list:
            tag_song_list.append([tag, song])

In [1597]:
result = train.apply(lambda x : co_occur_tag_song(x.songs, x.tags), axis=1)

In [1598]:
co_tag_song_df = pd.DataFrame(tag_song_list, columns=['tag', 'song'])

In [1599]:
co_tag_song_df_indexed =  co_tag_song_df.set_index('tag')

In [1600]:
co_tag_song_count_df = co_tag_song_df.groupby(['tag', 'song']).size().reset_index()

In [1601]:
co_tag_song_count_df.columns = ['tag', 'song', 'count']

In [1602]:
co_tag_song_count_df['count_scale'] = co_tag_song_count_df['count'].apply(lambda x : math.log(x+1))

In [1603]:
co_tag_song_count_df_indexed = co_tag_song_count_df.set_index('song')

In [1604]:
pp_tag_count_df = co_tag_song_df.groupby('tag').size().sort_values(ascending=False).reset_index().set_index('tag')

In [1605]:
song_has_tag_set = set(co_tag_song_count_df_indexed.index.tolist())

In [1606]:
tag_tag_list = []

In [1607]:
def co_occur_tag(tag_list):
    for tag1 in tag_list:
        for tag2 in tag_list:
            if tag1 is not tag2:
                tag_tag_list.append([tag1, tag2])


In [1608]:
result = train['tags'].apply(co_occur_tag)

In [1609]:
co_tag_df = pd.DataFrame(tag_tag_list, columns=['ta', 'tb'])

In [1610]:
co_tag_count_df = co_tag_df.groupby(['ta', 'tb']).size().reset_index().set_index('ta')

In [1611]:
co_tag_count_df.columns = ['tb', 'count']

In [1612]:
co_tag_count_df['count_scale'] = co_tag_count_df['count'].apply(lambda x : math.log(x+1))

In [1613]:
pp_tag_count_df = co_tag_df.groupby('ta').size().sort_values(ascending=False).reset_index().set_index('ta')

In [1614]:
tag_has_tag_set = set(co_tag_count_df.index.tolist())

In [1615]:
def get_reco_tag_co_occur(tag_list, size=10):
    if len(tag_list) == 0:
        return []
    
    tag_set = tag_has_tag_set & set(tag_list)
    co_tag = co_tag_count_df.loc[tag_set]
    co_tag_count = co_tag.groupby('tb')['count_scale'].sum().sort_values(ascending=False).reset_index()
    #print(co_tag_count)
    return co_tag_count[~co_tag_count['tb'].isin(tag_list)].head(size)

In [1616]:
def reco_tag_co_occur(tag_list, size=10):
    if len(tag_list) == 0:
        return []
    reco_top = get_reco_tag_co_occur(tag_list)
    if len(reco_top) == 0:
        return []
    return reco_top['tb'].tolist()

In [1617]:
val.iloc[21242]['tags']

['힙합', '우기', '비트', 'aomg']

In [1618]:
reco_tag_co_occur(val.iloc[14149]['tags'])

['크리스마스', '캐롤', '겨울', '연말', 'Christmas', '크리스마스캐롤', '눈', '기분전환', '팝', 'Carol']

In [1619]:
def get_reco_tag_song_co_occur(song_list, ignore_list=[], size=10):
    if len(song_list) == 0:
        return []
    
    song_set = song_has_tag_set & set(song_list)
    co_tag = co_tag_song_count_df_indexed.loc[song_set]
    co_tag_count = co_tag.groupby('tag')['count_scale'].sum().sort_values(ascending=False).reset_index()
    #print(co_tag_count.shape)
    #print(co_tag_count)
    return co_tag_count[~co_tag_count['tag'].isin(ignore_list)].head(size)

In [1620]:
def reco_tag_song_co_occur(song_list, ignore_list=[], size=10):
    if len(song_list) == 0:
        return []
    reco_top = get_reco_tag_song_co_occur(song_list, ignore_list)
    if len(reco_top) == 0:
        return []
    return reco_top['tag'].tolist()

In [1621]:
reco_tag_song_co_occur(val.iloc[14149]['songs'], val.iloc[14149]['tags'])

['크리스마스', '겨울', '캐롤', '기분전환', '연말', '눈', '사랑', '감성', 'Christmas', '카페']

In [1622]:
def reco_tag_song_mix(tag_list, song_list, size=10):
    reco_tag = get_reco_tag_co_occur(tag_list, 20)
    if len(reco_tag) == 0:
        return []
    reco_tag.columns = ['tag', 'count_tag']
    
    reco_song = get_reco_tag_song_co_occur(song_list, tag_list, 20)
    
    if len(reco_song) == 0:
        return reco_tag['tag'].tolist()
    
    reco_song.columns = ['tag', 'count_song']     
    reco = pd.merge(reco_tag, reco_song, how='outer', on='tag')
    reco = reco.fillna(0)
    reco['count'] = reco.apply(lambda x : x.count_tag + x.count_song * 0.1 , axis=1)
    reco = reco.sort_values('count', ascending=False)  
    #print(reco)
    return reco.head(size)['tag'].tolist()

In [1623]:
reco_tag_song_mix(val.iloc[14149]['tags'], val.iloc[14149]['songs'])

['크리스마스', '캐롤', '겨울', '연말', 'Christmas', '기분전환', '눈', '팝', '12월', '카페']

In [1624]:
pp_tag_count_top = pp_tag_count_df.head(100)

In [1625]:
reco_tag_count = 0
def tag_reco_final(song_list, tag_list):
    
    global reco_tag_count
    if reco_tag_count % 1000 == 0:
        print(f'===== working at {reco_tag_count},  {datetime.now()} =====')

    reco = reco_tag_co_occur(tag_list)
    #print(f'r1 : {len(reco)}')
    extra = 10 - len(reco)
    if extra > 0:
        reco.extend(reco_tag_song_co_occur(song_list, reco+tag_list, extra))
        #print(f'r2 : {len(reco)}')
    extra = 10 - len(reco)
    if extra > 0:
        reco.extend(pp_tag_count_df[~pp_tag_count_df.index.isin(reco+tag_list)].head(extra).index.tolist())
        #print(f'r3 : {len(reco)}')
    reco_tag_count = reco_tag_count + 1
    return reco[:10]

In [1626]:
reco_tag_count = 0
def tag_reco_final(song_list, tag_list):
    global reco_tag_count
    if reco_tag_count % 1000 == 0:
        print(f'===== working at {reco_tag_count},  {datetime.now()} =====')
        
    song_len = len(song_list)
    tag_len = len(tag_list)
    reco = []
    if song_len > 0 and tag_len > 0:
        reco = reco_tag_song_mix(tag_list, song_list)
    elif song_len > 0 and tag_len == 0:
        reco = reco_tag_song_co_occur(song_list)
    elif song_len == 0 and tag_len > 0:
        reco = reco_tag_co_occur(tag_list)
    
    extra = 10 - len(reco)
    if extra > 0:
        reco.extend(pp_tag_count_top[~pp_tag_count_top.index.isin(reco+tag_list)].head(extra).index.tolist())
    reco_tag_count = reco_tag_count + 1
    return reco[:10]

In [1627]:
#val['tag_reco'] = val.apply(lambda x : tag_reco_final(x.songs, x.tags), axis=1)
#val[['id', 'tag_reco']].to_csv('tag_reco_result.csv', encoding='utf-8-sig')

In [ ]:
test['tag_reco'] = test.apply(lambda x : tag_reco_final(x.songs, x.tags), axis=1)

===== working at 0,  2020-07-22 12:52:15.796093 =====
===== working at 1000,  2020-07-22 13:04:31.240137 =====
===== working at 2000,  2020-07-22 13:16:48.354874 =====
===== working at 3000,  2020-07-22 13:28:54.115111 =====
===== working at 4000,  2020-07-22 13:41:09.990813 =====
===== working at 5000,  2020-07-22 13:53:25.907667 =====


In [ ]:
test[['id', 'tag_reco']].to_csv('tag_reco_result.csv', encoding='utf-8-sig')

In [ ]:
val[(val['songs_len'] != 0) & (val['tags_len'] != 0)].sample(30)

In [ ]:
def get_reco_song_co_occur(song_list, ignore_list=[], size=100):
    reco_list = []
    for song in song_list:
        if song in co_song_count_df.index:
            reco_s = co_song_count_df.loc[song]['sb'].tolist()
            reco_c = co_song_count_df.loc[song]['count_scale'].tolist()
            if type(reco_s) is not list:
                reco_list.append([reco_s, reco_c])
            else:
                for idx, reco in enumerate(reco_s):
                    reco_list.append([reco_s[idx], reco_c[idx]])
    reco_df = pd.DataFrame(reco_list, columns=['song', 'count_scale'])
    #print(reco_df.head(20))
    reco_count_df = reco_df.groupby(['song'])['count_scale'].sum().sort_values(ascending=False).reset_index()
    #print(reco_count_df.head(20))
    reco_top = reco_count_df[~reco_count_df['song'].isin(song_list+ignore_list)].head(size)
    #print(reco_top.head(20))

    return reco_top

In [ ]:
def reco_song_co_occur(song_list, size=100):
    if len(song_list) == 0:
        return []
    reco_top = get_reco_song_co_occur(song_list)
    if len(reco_top) == 0:
        return []
    return reco_top['song'].tolist()

In [ ]:
song_index = 21242

In [ ]:
reco_result = reco_song_co_occur(val.iloc[song_index]['songs'])

In [ ]:
len(set(reco_result))

In [ ]:
len(set(reco_result) & set(val.iloc[song_index]['songs']))

In [ ]:
song_meta_indexed.loc[val.iloc[song_index]['songs']]

In [ ]:
song_meta_indexed.loc[reco_result].head(10)

In [ ]:
def get_reco_song_w2v(song_list, ignore_list=[], size=100):
    reco_num_per_song = int(size / len(song_list)) + 1
    reco_num_per_song = max(reco_num_per_song, 10) 
    reco_list = []
    
    for idx, song in enumerate(song_list):
        if str(song) in model.wv.vocab:
            reco_list.extend(model.wv.most_similar(str(song), topn=reco_num_per_song))
    #print(reco_list)
        
    reco_df = pd.DataFrame(reco_list, columns=['song', 'dist'])
    reco_count_df = reco_df.groupby('song')['dist'].sum().sort_values(ascending=False).reset_index()
    reco_count_df['song_id'] = reco_count_df['song'].apply(lambda x : int(x))
    reco_top = reco_count_df[~reco_count_df['song_id'].isin(song_list+ignore_list)].head(size)

    #print(reco_top[['song_id', 'dist']])
    return reco_top[['song_id', 'dist']]

In [ ]:
def reco_song_w2v(song_list, ignore_list=[], size=100):
    if len(song_list) == 0:
        return []
    reco_top = get_reco_song_w2v(song_list, ignore_list, size)
    if len(reco_top) == 0:
        return []
    return reco_top['song_id'].tolist()

In [ ]:
song_index = 21242


In [ ]:
reco_result = reco_song_w2v(val.iloc[song_index]['songs'])

In [ ]:
len(set(reco_result))

In [ ]:
len(set(reco_result) & set(val.iloc[song_index]['songs']))

In [ ]:
song_meta_indexed.loc[val.iloc[song_index]['songs']]

In [ ]:
song_meta_indexed.loc[reco_result].head(10)

In [ ]:
def get_reco_song_tag_co_occur(tag_list, ignore_list=[], size=100):
    tag_song_df = co_tag_song_count_df[co_tag_song_count_df['tag'].isin(tag_list)]
    #print(tag_song_df)
    if tag_song_df.shape[0] == 0:
        return []
    tag_song_count_df = tag_song_df.groupby('song')['count_scale'].sum().reset_index()
    #print(tag_song_count_df)
    tag_song_count_df_remove_ignore = tag_song_count_df[~tag_song_count_df['song'].isin(ignore_list)]
   
    tag_song_count_top = tag_song_count_df_remove_ignore.sort_values('count_scale', ascending=False).head(size)
    #print(tag_song_count_top.head(20))
    return tag_song_count_top

In [ ]:
def reco_song_tag_co_occur(tag_list, ignore_list=[], size=100):
    tags = list(set(tag_list))
    if len(tags) == 0:
        return []
    reco_top = get_reco_song_tag_co_occur(tags, ignore_list, size)
    if len(reco_top) == 0:
        return []
    return reco_top['song'].tolist()

In [ ]:
song_index = 100

In [ ]:
reco_result = reco_song_tag_co_occur(val.iloc[song_index]['tags'])

In [ ]:
len(set(reco_result))

In [ ]:
val.iloc[song_index]['tags']

In [ ]:
song_meta_indexed.loc[reco_result]

In [ ]:
def reco_song_co_occur_album_artist_count(song_list):
    #print(song_list)
    reco_ss = get_reco_song_co_occur(song_list, [], 200)
    
    if len(reco_ss) == 0:
        return []
    
    reco_ss.columns = ['song', 'count_song']
    reco_ss = pd.merge(reco_ss, song_album_df, how='left', left_on='song', right_on='song')
    #print(reco_ss)
    
    song_album_count = song_album_df.loc[song_list].groupby('album').size().reset_index()
    song_album_count.columns = ['album', 'count']
    #print(song_album_count)
    song_album_count = song_album_count.set_index('album')
       
    def get_album_count(album):
        count = 1
        if album in song_album_count.index:
            #print(song_album_count.loc[album]['count'])
            count = count + song_album_count.loc[album]['count']
        return math.log(count+1)
    reco_ss['count_album'] = reco_ss['album'].apply(get_album_count)
   

    reco_ss = pd.merge(reco_ss, song_meta[['id', 'artist_name_basket']], how='left', left_on='song', right_on='id')
    #print(reco_ss)
    
    song_artist_count = song_artist_df.loc[song_list].groupby('artist').size().reset_index()
    song_artist_count.columns = ['artist', 'count']
    #print(song_artist_count)
    song_artist_count = song_artist_count.set_index('artist')
       
    def get_artist_count(artist_list):    
        count = 1
        if len(artist_list) == 0:
            return count
        for artist in artist_list:
            if artist in song_artist_count.index:
                #print(song_artist_count.loc[artist]['count'])
                count = count + song_artist_count.loc[artist]['count']
        return math.log(max(int(count / len(artist_list)), 1)+1)
    reco_ss['count_artist'] = reco_ss['artist_name_basket'].apply(get_artist_count)
    
    
    reco_ss = pd.merge(reco_ss, song_meta[['id', 'song_gn_dtl_gnr_basket']], how='left', left_on='id', right_on='id')
    #print(reco_ss)
    song_gn_count = song_gn_df.loc[song_list].groupby('gn').size().reset_index()
    song_gn_count.columns = ['gn', 'count']
    #print(song_artist_count)
    song_gn_count = song_gn_count.set_index('gn')
    
    def get_gn_count(gn_list):
        count = 1
        if len(gn_list) == 0:
            return count
        for gn in gn_list:
            if gn in song_gn_count.index:
                #print(song_gn_count.loc[gn]['count'])
                count = count + song_gn_count.loc[gn]['count']
        return math.log(max(int(count / len(gn_list)), 1))
    reco_ss['count_gn'] = reco_ss['song_gn_dtl_gnr_basket'].apply(get_gn_count)

    reco = reco_ss
    reco['count'] = reco.apply(lambda x : x.count_song + x.count_album * 5 + x.count_artist * 5 + x.count_gn, axis=1)
    reco = reco.sort_values('count', ascending=False)  
    #print(reco.head(10))
    return reco.head(100)['song'].tolist()

In [ ]:
song_index = 30

In [ ]:
song_meta_indexed.loc[val.iloc[song_index]['songs']]

In [ ]:
reco_result = reco_song_co_occur_album_artist_count(val.iloc[song_index]['songs'])

In [ ]:
song_meta_indexed.loc[reco_result].head(10)

In [ ]:
pp_song_count_df_top200 = pp_song_count_df.head(200)

In [ ]:
reco_song_count = 0

def song_reco_final(song_list, tag_list):
    global reco_song_count
    song_len = len(song_list)
    tag_len = len(tag_list)

    reco = []
    extra = 100
    
    if reco_song_count % 1000 == 0:
        print(f'===== working at {reco_song_count},  {datetime.now()} =====')
    
    if song_len > 0 and tag_len > 0:
        reco = reco_song_co_occur_album_artist_count(song_list)
        extra = 100 - len(reco)
        if extra > 0:
            reco.extend(reco_song_w2v(song_list, reco, extra))
        extra = 100 - len(reco)
        if extra > 0:
            reco.extend(reco_song_tag_co_occur(tag_list, song_list+reco, extra))
    elif song_len > 0 and tag_len == 0:
        reco = reco_song_co_occur_album_artist_count(song_list)
        extra = 100 - len(reco)
        if extra > 0:
            reco.extend(reco_song_w2v(song_list, reco, extra))
    elif song_len == 0 and tag_len > 0:
        reco = reco_song_tag_co_occur(tag_list)
    
    extra = 100 - len(reco)
    if extra > 0:
        reco.extend(pp_song_count_df_top200[~pp_song_count_df_top200.index.isin(song_list+reco)].head(extra).index.tolist())
    reco_song_count = reco_song_count + 1
    return reco[:100]

In [ ]:
#val['song_reco'] = val.apply(lambda x : song_reco_final(x.songs, x.tags), axis=1)
#val[['id', 'song_reco']].to_csv('song_reco_result.csv')

In [ ]:
test['song_reco'] = test.apply(lambda x : song_reco_final(x.songs, x.tags), axis=1)
test[['id', 'song_reco']].to_csv('song_reco_result.csv')

In [ ]:
song_reco = pd.read_csv('song_reco_result.csv')

In [ ]:
song_reco['song_reco'] = song_reco['song_reco'].apply(lambda x : eval(x))

In [ ]:
#val['song_reco'] = song_reco['song_reco']
test['song_reco'] = song_reco['song_reco']

In [ ]:
tag_reco = pd.read_csv('tag_reco_result.csv')

In [ ]:
tag_reco['tag_reco'] = tag_reco['tag_reco'].apply(lambda x : eval(x))

In [ ]:
#val['tag_reco'] = tag_reco['tag_reco']
test['tag_reco'] = tag_reco['tag_reco']

In [ ]:
answers = []

def make_answer(pid, songs, tags):
    answers.append({
        "id": pid,
        "songs": songs,
        "tags": tags
    }) 

In [ ]:
#result = val.apply(lambda x : make_answer(x.id, x.song_reco, x.tag_reco), axis=1)
result = test.apply(lambda x : make_answer(x.id, x.song_reco, x.tag_reco), axis=1)

In [ ]:
import io
import os
import json
import distutils.dir_util

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
write_json(answers, 'results.json')